In [29]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_defense = client.open("frb-volley-game-stats").worksheet("player-defense")
player_positions = client.open("frb-volley-game-stats").worksheet("player-positions")
player_reception = client.open("frb-volley-game-stats").worksheet("player-reception")

# Fetch all records (rows) from the sheet
player_defense_data = player_defense.get_all_records()
player_reception_data = player_reception.get_all_records()


In [30]:
# Print the data or convert it into a DataFrame
import pandas as pd
player_positions_df = pd.DataFrame(player_positions.get_all_records())
df = pd.DataFrame(player_defense_data)
df['pass-rating'] = pd.to_numeric(df['pass-rating'], errors='coerce')
df['pass-attempt'] = pd.to_numeric(df['pass-attempt'])
df['pass-attempt'] = df['pass-attempt'].fillna(0).astype(int)

df['pass-error'] = pd.to_numeric(df['pass-error'])
df['pass-error'] = df['pass-error'].fillna(0).astype(int)

df['digs'] = pd.to_numeric(df['digs'])
df['digs'] = df['digs'].fillna(0).astype(int)

df['dig-error'] = pd.to_numeric(df['dig-error'])
df['dig-error'] = df['dig-error'].fillna(0).astype(int)

df['blocks'] = pd.to_numeric(df['blocks'])
df['blocks'] = df['blocks'].fillna(0).astype(int)

df['block-error'] = pd.to_numeric(df['block-error'])
df['block-error'] = df['block-error'].fillna(0).astype(int)


#df["error-pct"] = df["attack-errors"] / df["attack"]
#df["pass-rating"].unique()
df

,date,match-id,match,set,player,pass-attempt,pass-error,pass-rating,digs,dig-error,blocks,block-error,defense-errors
0,08/12/2024,141108,frb-gentofte,1,Lasse,0,0,NaN,1,1,0,2,3
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,NaN,0,0,0,0,0
2,08/12/2024,141108,frb-gentofte,1,Kristian,0,0,NaN,0,0,0,0,0
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,NaN,0,0,0,0,0
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,29/03/2025,141144,hvidovre-frb,3,Mads,0,0,NaN,0,0,0,0,
744,29/03/2025,141144,hvidovre-frb,3,Martin,0,0,NaN,0,0,0,0,
745,29/03/2025,141144,hvidovre-frb,3,Mikkel,0,0,NaN,0,0,0,0,
746,29/03/2025,141144,hvidovre-frb,3,Frederik,0,0,NaN,0,0,0,0,


In [31]:

# Group by player to get player-level stats
player_stats = df.groupby('player').agg(
    avg_pass_rating=('pass-rating', 'mean'),
    number_pass_attempts=('pass-attempt', 'sum')
).reset_index()

player_stats

#combined_avg_pass_rating = player_stats['avg_pass_rating'].mean()
#combined_avg_digs_per_game = game_stats['total_digs'].mean()

#combined_avg_pass_rating

,player,avg_pass_rating,number_pass_attempts
0,Alex,10.784000,97
1,Ando,0.000000,0
2,Bo,0.000000,0
3,Boerme,12.378462,41
4,Bosse,36.333333,8
5,Frederik,0.190000,7
6,Gustav,0.428571,3
7,Kristian,0.166667,1
8,Lasse,0.000000,0
9,Mads,0.000000,0


In [4]:
player_positions_df

,player,position,passer
0,Lasse,setter,0
1,Nico,outside,1
2,Kristian,middle,0
3,Ando,middle,0
4,Bosse,outside,1
5,Boerme,outside,1
6,Bo,middle,0
7,Soeren,opposite,0
8,Nicola,outside,1
9,Vestbjerg,outside,1


In [32]:
passer_stats = pd.merge(player_stats, player_positions_df, on="player")
passer_stats = passer_stats[passer_stats["passer"] == 1]
passer_stats

,player,avg_pass_rating,number_pass_attempts,position,passer
0,Alex,10.784000,97,libero,1
3,Boerme,12.378462,41,outside,1
4,Bosse,36.333333,8,outside,1
5,Frederik,0.190000,7,outside,1
11,Mikkel,27.166667,8,outside,1
12,Nico,0.500000,2,outside,1
13,Nicola,12.032000,67,outside,1
15,Vestbjerg,1.086000,34,outside,1


In [33]:
df_player_reception = pd.DataFrame(player_reception_data)
#df_player_reception['weighted_rating'] = df_player_reception['pass-rating'] * df_player_reception['pass-attempt']
df_player_reception

,date,match-id,match,set,player,pass-attempt,pass-rating,pass-error,pass-1,pass-2,pass-3,overpass-in-play
0,08/12/2024,141108,frb-gentofte,1,Lasse,0,0.0,0,0,0,0,0
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0.0,0,0,0,0,0
2,08/12/2024,141108,frb-gentofte,1,Kristian,0,0.0,0,0,0,0,0
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0.0,0,0,0,0,0
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
743,29/03/2025,141144,hvidovre-frb,3,Mads,0,0.0,0,0,0,0,0
744,29/03/2025,141144,hvidovre-frb,3,Martin,0,0.0,0,0,0,0,0
745,29/03/2025,141144,hvidovre-frb,3,Mikkel,0,0.0,0,0,0,0,0
746,29/03/2025,141144,hvidovre-frb,3,Frederik,0,0.0,0,0,0,0,0


In [34]:


# Group by player to get player-level stats
player_reception_stats = df_player_reception.groupby('player').agg(
    avg_pass_rating=('pass-rating', 'mean'),
    number_pass_attempts=('pass-attempt', 'sum'),
    positive_pct=('pass-2', 'mean')
).reset_index()

player_reception_stats

,player,avg_pass_rating,number_pass_attempts,positive_pct
0,Alex,1.645378,228,1.571429
1,Ando,0.000000,0,0.000000
2,Bo,0.040816,2,0.020408
3,Boerme,0.777381,84,0.612245
4,Bosse,0.245748,21,0.122449
5,Frederik,0.036866,7,0.032258
6,Gustav,0.233236,12,0.061224
7,Kristian,0.204082,5,0.020408
8,Lasse,0.020408,1,0.000000
9,Mads,0.000000,0,0.000000


In [35]:
df_player_reception

rating_summary = df_player_reception.groupby('player').agg(
    average_pass_rating=('pass-rating', 'mean')
).reset_index()
rating_summary

,player,average_pass_rating
0,Alex,1.645378
1,Ando,0.000000
2,Bo,0.040816
3,Boerme,0.777381
4,Bosse,0.245748
5,Frederik,0.036866
6,Gustav,0.233236
7,Kristian,0.204082
8,Lasse,0.020408
9,Mads,0.000000


In [36]:
# Group by player and sum all relevant columns
summary = df_player_reception.groupby('player').sum().reset_index()

# Calculate positive and perfect pass counts
summary['positive_passes'] = summary['pass-2'] + summary['pass-3']
summary['perfect_passes'] = summary['pass-3']

# Calculate percentages
summary['positive_percentage'] = (summary['positive_passes'] / summary['pass-attempt']) * 100
summary['perfect_percentage'] = (summary['perfect_passes'] / summary['pass-attempt']) * 100
summary['error_percentage'] = (summary['pass-error'] / summary['pass-attempt']) * 100

# Show final summary
summary = summary[['player', 'pass-attempt', 'positive_percentage', 'perfect_percentage', "error_percentage"]]

final_summary = pd.merge(
    summary[['player', 'pass-attempt', 'positive_percentage', 'perfect_percentage', 'error_percentage']],
    rating_summary,
    on='player'
)

summary = pd.merge(final_summary, player_positions_df, on="player")
summary = summary[summary["passer"] == 1]
summary = summary[["player", 'pass-attempt',"error_percentage", "positive_percentage", "perfect_percentage", "average_pass_rating"]]
reception_summary = summary[summary['positive_percentage'].notna()]
reception_summary

,player,pass-attempt,error_percentage,positive_percentage,perfect_percentage,average_pass_rating
0,Alex,228,7.017544,66.228070,32.456140,1.645378
3,Boerme,84,8.333333,66.666667,30.952381,0.777381
4,Bosse,21,9.523810,52.380952,23.809524,0.245748
5,Frederik,7,28.571429,28.571429,14.285714,0.036866
11,Mikkel,62,6.451613,66.129032,14.516129,0.629301
12,Nico,4,0.000000,75.000000,25.000000,0.122449
13,Nicola,200,3.500000,69.000000,29.000000,1.426076
15,Vestbjerg,226,9.292035,59.734513,22.123894,1.156270


In [37]:
import json

# Convert DataFrame to JSON
data = reception_summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-passing-summary.json", "w") as f:
    json.dump(data, f, indent=4)